In [2]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
%matplotlib inline

import seaborn as sns

from bokeh.io import output_notebook, show
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource
from bokeh.models.tools import HoverTool
from bokeh.models import Range1d
from bokeh.models.markers import Cross
from bokeh.models.glyphs import Quad
from bokeh.models.glyphs import Line

output_notebook()

Loading BokehJS ...

In [3]:

def read_att(path):
    '''
    Used to read the Vissim Attribute file for Node evaluation results 
    Returns a dataframe containing the results
    Based on the default layout for node results
    '''
    att = pd.read_table(path, sep = ";", header=28)

    att.columns = att.columns.str.replace('(', '')
    att.columns = att.columns.str.replace(')', '')
    att.columns = att.columns.str.replace('\\', ' ')
    att = att[att['$MOVEMENTEVALUATION:SIMRUN'] == 'AVG']
    att = att[att['VEHSALL'] != 0]
    att['LOSCOL'] = att.LOSVALALL.replace([1, 2, 3, 4, 5, 6], 
                                          ['#44AAFF', '#44FFEE', '#AAFF44', '#FFEE44', '#FFAA33', '#FF4433'])
    att['LOSALL'] = att.LOSVALALL.replace([1, 2, 3, 4, 5, 6], 
                                          ['LOS_A', 'LOS_B', 'LOS_C', 'LOS_D', 'LOS_E', 'LOS_F'])
    att = att.sort_values(by=['LOSVALALL'])

    
    return att

def read_tt_att(path, scen):
    '''
    Used to read the Vissim Attribute file for Vehicle Travel Time results 
    Returns a dataframe containing the resuluts
    Based on the default layout for Travel Time results
    '''
    att = pd.read_table(path, sep = ";", header=17)

    att.columns = att.columns.str.replace('(', '')
    att.columns = att.columns.str.replace(')', '')
    att.columns = att.columns.str.replace('\\', ' ')
    att = att[att['$VEHICLETRAVELTIMEMEASUREMENTEVALUATION:SIMRUN'] == 'AVG']
    att = att[att['VEHSALL'] != 0]
    att = att[att['VEHICLETRAVELTIMEMEASUREMENT'] > 20]
    att.DISTTRAVALL = att.DISTTRAVALL.cumsum()
    att.TRAVTMALL = att.TRAVTMALL.cumsum()
    att['SCENARIO'] = scen
    
    return att

In [4]:

def LOS_plot(attdf, scen, attdf2 = []):
    '''
    Creates a graph displaying the average delay vs the volume per movement for all movements in model
    Categorised by Levl of Service
    '''
    source = ColumnDataSource(attdf)
    
    p = figure(width=600, height=600)
    p.circle(x='VEHSALL', y='VEHDELAYALL', source=source, size=10, line_color='black', fill_color = 'LOSCOL', alpha=0.7, legend='LOSALL')
    
    if not attdf2.empty:
        source2 = ColumnDataSource(attdf2)
        glyph = Cross(x='VEHSALL', y='VEHDELAYALL', 
                      size=10, line_color='LOSCOL', 
                      fill_color=None, line_width=1.7, 
                      line_alpha = 0.5
                     )
        p.add_glyph(source2, glyph)
        
    hover = HoverTool()
    hover.tooltips=[
        ('Node', '@MOVEMENT NODE NAME'),
        ('Movement', '@MOVEMENT'),
        ('Volume', '@VEHSALL'),
        ('Delay', '@VEHDELAYALL')
    ]

    p.add_tools(hover)

    p.title.text = 'Delay vs Volume per Movement by Level of Service'
    p.xaxis.axis_label = 'Volume (Vehicles)'
    p.yaxis.axis_label = 'Delay (s)'
    p.background_fill_color = "#9B9893"
    p.background_fill_alpha = 0.5
    p.legend.location = "top_right"
    
    return p


In [5]:

def QLEN_dist_plot(attdf, bins, scen, scen2 = '', attdf2 = []):
    
    title = 'Distribution of Queue Lengths in Model'
    
    measured = attdf.QLEN
    hist, edges = np.histogram(measured, density=False, bins = bins)

    p = figure(title=title, background_fill_color="#fafafa",
              width=650, height=350)
    p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
           fill_color="navy", line_color="white", alpha=0.5, legend = scen.split('-')[1])
    
    if not attdf2.empty:
        measured2 = attdf2.QLEN
        hist2, edges2 = np.histogram(measured2, density=False, bins = bins)
        p.quad(top=hist2, bottom=0, left=edges2[:-1], right=edges2[1:],
                fill_color="green", line_color="grey", alpha=0.5, legend = scen2.split('-')[1])
    
    p.y_range.start = 0
    p.legend.location = "center_right"
    p.legend.background_fill_color = "white"
    p.xaxis.axis_label = 'Length (m)'
    p.yaxis.axis_label = 'Number of Queues'
    p.grid.grid_line_color="white"
    
    hover = HoverTool()

    p.add_tools(hover)
    
    return p


In [9]:

def tt_plot(attdf, attdf2, scen1, scen2, attdf3 = pd.DataFrame(), scen3 =''):
    
    title = 'Travel time vs distance along Beach Road'
    
    source = ColumnDataSource(attdf)
    source2 = ColumnDataSource(attdf2)
    
    # Define the line glyph
    p = figure(title=title, background_fill_color="#fafafa",
              width=950, height=450)
        
    p.line(x='DISTTRAVALL', y='TRAVTMALL',
             line_width=5, line_color='#69995D', line_alpha=0.6,
             hover_line_color='green', hover_line_alpha=1.0,
             source=source, legend=scen1)
    p.line(x='DISTTRAVALL', y='TRAVTMALL',
             line_width=5, line_color='#394648', line_alpha=0.6,
             hover_line_color='green', hover_line_alpha=1.0,
             source=source2, legend=scen2)
    
    if not attdf3.empty:
        source3 = ColumnDataSource(attdf3)
        p.line(x='DISTTRAVALL', y='TRAVTMALL',
                 line_width=5, line_color='#ED8E6F', line_alpha=0.6,
                 hover_line_color='green', hover_line_alpha=1.0,
                 source=source3, legend=scen3)
    
    p.y_range.start = 0
    p.x_range.start = 0
    p.legend.location = "top_left"
    p.legend.background_fill_color = "white"
    p.xaxis.axis_label = 'Distance (m)'
    p.yaxis.axis_label = 'Travel Time (s)'
    p.grid.grid_line_color="black"
    
    hover = HoverTool()

    p.add_tools(hover)
    
    return p


In [80]:
run1 = '01'
scen1 = 'Beach Road Existing - Upgraded AM Future'
path1 = 'C:/Users/shafeeq.mollagee/Aurecon Group/Ilyaas Adams - 01 - Beach Road Complete/Results/%s_Node Results run %s.att' % (scen1, run1)
save_path1 = 'C:/Users/shafeeq.mollagee/Aurecon Group/Ilyaas Adams - 01 - Beach Road Complete/Results/%s.csv' % ('Nodes')

run2 = '02'
scen2 = 'Beach Road Existing - Upgraded AM 2017'
path2 = 'C:/Users/shafeeq.mollagee/Aurecon Group/Ilyaas Adams - 01 - Beach Road Complete/Results/%s_Node Results run %s.att' % (scen2, run2)


In [77]:
show(LOS_plot(read_att(path1), scen1, read_att(path2)))

In [ ]:
attg = att.groupby('LOSALL').MOVEMENT.count()
attg

In [81]:
show(QLEN_dist_plot(read_att(path2), 50, scen2, scen1, read_att(path1)))

In [7]:
run1 = '07'
scen1 = 'Beach Road Existing - Existing AM 2017'
path1 = 'C:/Users/shafeeq.mollagee/Aurecon Group/Ilyaas Adams - 01 - Beach Road Complete/Results/%s_Vehicle Travel Time Results run %s.att' % (scen1, run1)

run2 = '04'
scen2 = 'Beach Road Existing - Upgraded AM 2017'
path2 = 'C:/Users/shafeeq.mollagee/Aurecon Group/Ilyaas Adams - 01 - Beach Road Complete/Results/%s_Vehicle Travel Time Results run %s.att' % (scen2, run2)

run3 = '02'
scen3 = 'Beach Road Existing - Upgraded AM Future'
path3 = 'C:/Users/shafeeq.mollagee/Aurecon Group/Ilyaas Adams - 01 - Beach Road Complete/Results/%s_Vehicle Travel Time Results run %s.att' % (scen3, run3)

#run4 = '02'
#scen4 = 'Beach Road Existing - Upgraded AM 2017'
#path4 = 'C:/Users/shafeeq.mollagee/Aurecon Group/Ilyaas Adams - 01 - Beach Road Complete/Results/%s_Vehicle Travel Time Results run %s.att' % (scen2, run2)


In [11]:
show(tt_plot(read_tt_att(path1, scen1), read_tt_att(path2, scen2), scen1, scen2))